In [0]:
display(dbutils.fs.mounts())


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
dbutils.fs.mount(
  source = "wasbs://assignment2@21784980assignment2.blob.core.windows.net",
  mount_point = "/mnt/data",
  extra_configs = {"fs.azure.account.key.21784980assignment2.blob.core.windows.net":"k910kGxE8kl2wWaTtjyrd/uSUbHVxYL2/+UhaQ5Eaz8/ySBid5xOWXs4uTJIilygoCLxDqZFlaXZ+ASt26if4A=="})

Out[4]: True

In [0]:
display(dbutils.fs.mounts())


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/mnt/data,wasbs://assignment2@21784980assignment2.blob.core.windows.net,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
df_air_quality = spark.read.format("csv").option("header", "true").load("/mnt/data/Melbourne_Air_Quality_Data.csv")
df_air_quality.show(5)


+---------+-------------------+-------------+-----------------+-----+----+
sensor_id| timestamp| location|air_quality_index|pm2_5|pm10|
+---------+-------------------+-------------+-----------------+-----+----+
 AQ1|2024-09-01 00:00:00|Melbourne CBD| 45| 10.5|25.0|
 AQ2|2024-09-01 01:00:00| Fitzroy| 65| 18.0|35.5|
 AQ3|2024-09-01 02:00:00| Docklands| 58| 15.3|30.2|
 AQ4|2024-09-01 03:00:00| St Kilda| 72| 20.1|40.0|
 AQ5|2024-09-01 04:00:00| Footscray| 90| 25.0|50.1|
+---------+-------------------+-------------+-----------------+-----+----+

In [0]:
df_traffic = spark.read.format("csv").option("header", "true").load("/mnt/data/Melbourne_Traffic_Data.csv")
df_energy = spark.read.format("csv").option("header", "true").load("/mnt/data/Melbourne_Energy_Consumption_Data.csv")

df_traffic.show(5)
df_energy.show(5)


+---------+-------------------+-----------------+-------------+-------------+
sensor_id| timestamp| location|vehicle_count|avg_speed_kmh|
+---------+-------------------+-----------------+-------------+-------------+
 T1|2024-09-01 00:00:00| Eastern Freeway| 1000| 80|
 T2|2024-09-01 01:00:00| Melbourne CBD| 800| 30|
 T3|2024-09-01 02:00:00|West Gate Freeway| 1200| 60|
 T4|2024-09-01 03:00:00| St Kilda Road| 600| 50|
 T5|2024-09-01 04:00:00| Monash Freeway| 1500| 70|
+---------+-------------------+-----------------+-------------+-------------+

+---------+-------------------+--------------------+-------------------+------------+
sensor_id| timestamp| location|energy_consumed_kwh|peak_load_mw|
+---------+-------------------+--------------------+-------------------+------------+
 E1|2024-09-01 00:00:00|Residential - Sou...| 200| 12|
 E2|2024-09-01 01:00:00|Commercial - Melb...| 1000| 35|
 E3|2024-09-01 02:00:00|Industrial - Lave...| 1200| 50|
 E4|2024-09-01 03:00:00| Richmond| 500| 20|
 E5|2024-09-01 04:00:00| Essendon| 600| 25|
+---------+-------------------+--------------------+-------------------+------------+

In [0]:
# Check if data is loaded correctly
df_air_quality.show(5)
df_traffic.show(5)
df_energy.show(5)


+---------+-------------------+-------------+-----------------+-----+----+
sensor_id| timestamp| location|air_quality_index|pm2_5|pm10|
+---------+-------------------+-------------+-----------------+-----+----+
 AQ1|2024-09-01 00:00:00|Melbourne CBD| 45| 10.5|25.0|
 AQ2|2024-09-01 01:00:00| Fitzroy| 65| 18.0|35.5|
 AQ3|2024-09-01 02:00:00| Docklands| 58| 15.3|30.2|
 AQ4|2024-09-01 03:00:00| St Kilda| 72| 20.1|40.0|
 AQ5|2024-09-01 04:00:00| Footscray| 90| 25.0|50.1|
+---------+-------------------+-------------+-----------------+-----+----+

+---------+-------------------+-----------------+-------------+-------------+
sensor_id| timestamp| location|vehicle_count|avg_speed_kmh|
+---------+-------------------+-----------------+-------------+-------------+
 T1|2024-09-01 00:00:00| Eastern Freeway| 1000| 80|
 T2|2024-09-01 01:00:00| Melbourne CBD| 800| 30|
 T3|2024-09-01 02:00:00|West Gate Freeway| 1200| 60|
 T4|2024-09-01 03:00:00| St Kilda Road| 600| 50|
 T5|2024-09-01 04:00:00| Monash Freeway| 1500| 70|
+---------+-------------------+-----------------+-------------+-------------+

+---------+-------------------+--------------------+-------------------+------------+
sensor_id| timestamp| location|energy_consumed_kwh|peak_load_mw|
+---------+-------------------+--------------------+-------------------+------------+
 E1|2024-09-01 00:00:00|Residential - Sou...| 200| 12|
 E2|2024-09-01 01:00:00|Commercial - Melb...| 1000| 35|
 E3|2024-09-01 02:00:00|Industrial - Lave...| 1200| 50|
 E4|2024-09-01 03:00:00| Richmond| 500| 20|
 E5|2024-09-01 04:00:00| Essendon| 600| 25|
+---------+-------------------+--------------------+-------------------+------------+

In [0]:
from pyspark.sql.functions import regexp_replace, trim

# Standardize location names
# For example, remove extra descriptors like 'Residential', 'Commercial', etc.

# Air Quality Dataset
df_air_quality = df_air_quality.withColumn("location", trim(df_air_quality.location))

# Traffic Dataset
df_traffic = df_traffic.withColumn("location", trim(df_traffic.location))

# Energy Dataset: Standardize by removing 'Residential - ', 'Commercial - ', etc.
df_energy = df_energy.withColumn("location", regexp_replace("location", "Residential - ", ""))
df_energy = df_energy.withColumn("location", regexp_replace("location", "Commercial - ", ""))
df_energy = df_energy.withColumn("location", regexp_replace("location", "Industrial - ", ""))
df_energy = df_energy.withColumn("location", trim(df_energy.location))

# Now the locations will be more standardized across datasets


In [0]:
# 1. Calculate average air quality index per region
avg_air_quality = df_air_quality.groupBy("location").agg({"air_quality_index": "avg"}).withColumnRenamed("avg(air_quality_index)", "avg_air_quality_index")

# 2. Calculate total traffic density (sum of vehicle counts) per region
traffic_density = df_traffic.groupBy("location").agg({"vehicle_count": "sum"}).withColumnRenamed("sum(vehicle_count)", "total_vehicle_count")

# 3. Calculate total energy consumption per region
total_energy_consumption = df_energy.groupBy("location").agg({"energy_consumed_kwh": "sum"}).withColumnRenamed("sum(energy_consumed_kwh)", "total_energy_consumed_kwh")

avg_air_quality.show()
traffic_density.show()
total_energy_consumption.show()

+-------------+---------------------+
 location|avg_air_quality_index|
+-------------+---------------------+
 Docklands| 58.0|
 St Kilda| 72.0|
 Fitzroy| 65.0|
Melbourne CBD| 45.0|
 Footscray| 90.0|
+-------------+---------------------+

+-----------------+-------------------+
 location|total_vehicle_count|
+-----------------+-------------------+
 St Kilda Road| 600.0|
West Gate Freeway| 1200.0|
 Eastern Freeway| 1000.0|
 Monash Freeway| 1500.0|
 Melbourne CBD| 800.0|
+-----------------+-------------------+

+-------------+-------------------------+
 location|total_energy_consumed_kwh|
+-------------+-------------------------+
 Laverton| 1200.0|
Melbourne CBD| 1000.0|
 Essendon| 600.0|
 Richmond| 500.0|
 South Yarra| 200.0|
+-------------+-------------------------+

Transfer to cosmos DB

In [0]:
# Set Cosmos DB connection details
cosmos_endpoint = "https://21784980assignmetn2.documents.azure.com:443/"
cosmos_master_key = "qbZhmeclbj4D3kZ0xyIYHfPEtoD618Qw2gFoVmO5B1345OPCq2MvKrztZ9bC0jQT9UB7Ss8raUmMACDbjZ9cGg=="
cosmos_database_name = "ToDoList"
cosmos_container_name = "21784980assignmetn2"

# Set the Spark configuration to access Cosmos DB
spark.conf.set("spark.cosmos.accountEndpoint", cosmos_endpoint)
spark.conf.set("spark.cosmos.accountKey", cosmos_master_key)
spark.conf.set("spark.cosmos.database", cosmos_database_name)
spark.conf.set("spark.cosmos.container", cosmos_container_name)


In [0]:
from pyspark.sql.functions import col, concat, lit, expr
import uuid
avg_air_quality = avg_air_quality.withColumn("id", expr("uuid()"))
# Writing structured data (summary_metrics) to Cosmos DB
avg_air_quality.write.format("cosmos.oltp").mode("append").options(
    **{
        "spark.cosmos.accountEndpoint": cosmos_endpoint,
        "spark.cosmos.accountKey": cosmos_master_key,
        "spark.cosmos.database": cosmos_database_name,
        "spark.cosmos.container": cosmos_container_name
    }
).save()



In [0]:
traffic_density = traffic_density.withColumn("id", expr("uuid()"))
# Writing structured data (summary_metrics) to Cosmos DB
traffic_density.write.format("cosmos.oltp").mode("append").options(
    **{
        "spark.cosmos.accountEndpoint": cosmos_endpoint,
        "spark.cosmos.accountKey": cosmos_master_key,
        "spark.cosmos.database": cosmos_database_name,
        "spark.cosmos.container": cosmos_container_name
    }
).save()


In [0]:
total_energy_consumption = total_energy_consumption.withColumn("id", expr("uuid()"))
# Writing structured data (summary_metrics) to Cosmos DB
total_energy_consumption.write.format("cosmos.oltp").mode("append").options(
    **{
        "spark.cosmos.accountEndpoint": cosmos_endpoint,
        "spark.cosmos.accountKey": cosmos_master_key,
        "spark.cosmos.database": cosmos_database_name,
        "spark.cosmos.container": cosmos_container_name
    }
).save()
